# Load Packages

In [1]:
import matplotlib.pyplot as plt
from numba import njit
import numpy as np

# Functions for the Markov Model

In [3]:
@njit(fastmath = True)
def transition_matrix(Vm: np.float64) -> np.ndarray[np.float64]:
    """Goal:
        This function returns the transition matrix of the Markov model for the
        Kv11 channel at a given voltage Vm.
        
        Adapted from the KV11.1 Markov model described in:
            Mazhari R, Greenstein JL, Winslow RL, Marbán E, Nuss HB. 
            Molecular interactions between two long-QT syndrome gene products, 
            HERG and KCNE2, rationalized by in vitro and in silico analysis. 
            Circ Res. 2001 Jul 6;89(1):33-8. doi: 10.1161/hh1301.093633. 
            PMID: 11440975.
    ---------------------------------------------------------------------------
    Input:
        Vm: Membrane voltage [mV]
    ---------------------------------------------------------------------------
    Output:
        Q: Transition matrix of the Markov model [s^-1]"""
    # Transition rates [ms^-1]
    # 5 States; 3 closed (C1, C2, C3), a transition state (I) and an open state (O)
    # C1 - C2 - C3 - O
    #            \  /
    #              I    
    C1C2 = 0.0069 * np.exp(0.0272 * Vm) # Symbol in the paper --> a0
    C2C1 = 0.0227 * np.exp(-0.0431 * Vm) # b0
    C3O = 0.0218 * np.exp(0.0262 * Vm) # a1
    OC3 = 0.0009 * np.exp(-0.0269 * Vm) # b1
    OI = 0.0622 * np.exp(0.0120 * Vm) # ai
    IO = 0.0059 * np.exp(-0.0443 * Vm) # bi
    C2C3 = 0.0266 # kf
    C3C2 = 0.1348 # kb
    C3I = 1.29E-5 * np.exp(2.71E-6 * Vm) # ai3
    IC3 = (OC3 * IO * C3I) / (C3O * OI) # Greek letter

    # Transition matrix
    Q = np.zeros((5, 5), dtype = np.float64)
    Q[0, 0] = -C1C2 # C1
    Q[0, 1] = C2C1
    Q[1, 0] = C1C2 # C2
    Q[1, 1] = -(C2C1 + C2C3)
    Q[1, 2] = C3C2
    Q[2, 2] = -(C3C2 + C3O + C3I) # C3
    Q[2, 1] = C2C3
    Q[2, 3] = OC3
    Q[2, 4] = IC3
    Q[3, 3] = -(OC3 + OI) # O
    Q[3, 2] = C3O
    Q[3, 4] = IO
    Q[4, 4] = -(IO + IC3) # I
    Q[4, 2] = C3I 
    Q[4, 3] = OI
    
    return Q * 1000 # ms^-1 to s^-1


def steady_state(Vm: np.float64) -> np.ndarray[np.float64]:
    """Goal:
        This function returns the steady state probabilities of the Markov model
        for the Kv11 channel at a given voltage Vm.
    ---------------------------------------------------------------------------
    Input:
        Vm: Membrane voltage [mV]
    ---------------------------------------------------------------------------
    Output:
        steady_state: Steady state probabilities of the Markov model
    """
    # Create transition matrix and adapt the last row in such a way that all
    # have weight 1
    Q = transition_matrix(Vm)
    Q[-1, :] = 1
    # Solve the linear system --> last equation ensures that sum of probabilities is 1
    b = np.array([0, 0, 0, 0, 1])
    steady_state = np.linalg.solve(Q, b)
    return steady_state

@njit(fastmath = True)
def kv11(t: np.float64, y: np.ndarray[np.float64], Vm: np.float64) -> np.ndarray[np.float64]:
    """Goal:
        This function returns the derivative of the state probabilities of the
        Kv11 channel at a given voltage Vm.
    ---------------------------------------------------------------------------
    Input:
        t: Time [s]
        y: State probabilities of the Markov model
        Vm: Membrane voltage [mV]
    ---------------------------------------------------------------------------
    Output:
        dydt: Derivative of the state probabilities of the Markov model
    """
    Q = transition_matrix(Vm)
    return Q @ y
